# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [1]:
# Your code goes here
import pandas as pd
# Dataset URL = https://bit.ly/3BPcobU
# Downloading the data
!wget -O crunchbase-investments.csv https://bit.ly/3BPcobU

--2023-05-08 05:42:27--  https://bit.ly/3BPcobU
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://archive.org/download/crunchbase-investments/crunchbase-investments.csv [following]
--2023-05-08 05:42:27--  https://archive.org/download/crunchbase-investments/crunchbase-investments.csv
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia904601.us.archive.org/28/items/crunchbase-investments/crunchbase-investments.csv [following]
--2023-05-08 05:42:28--  https://ia904601.us.archive.org/28/items/crunchbase-investments/crunchbase-investments.csv
Resolving ia904601.us.archive.org (ia904601.us.archive.org)... 207.241.235.61
Connecting to ia904601.us.archive.org (ia904601.us.archive.org)|207.241.235.61|:4

**Introductory tasks**

Read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory

In [2]:
# Create an iterable for iterating over the chunks
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')

In [3]:
# Check the memory usage of the first chunk
df_crunch = next(df_crunch_iter)

print(df_crunch.info(memory_usage='deep'))
# The printout below shows a memory usage for the chunck of 5.6 MB.

# It also shows how much memory each column utilizes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       5000 non-null   object 
 1   company_name            5000 non-null   object 
 2   company_category_code   4948 non-null   object 
 3   company_country_code    5000 non-null   object 
 4   company_state_code      4947 non-null   object 
 5   company_region          5000 non-null   object 
 6   company_city            4936 non-null   object 
 7   investor_permalink      5000 non-null   object 
 8   investor_name           5000 non-null   object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   4222 non-null   object 
 11  investor_state_code     3629 non-null   object 
 12  investor_region         5000 non-null   object 
 13  investor_city           4100 non-null   object 
 14  funding_round_type      5000 non-null   

In [4]:
# Check columns in the chunk with missing values

# df_crunch_numeric = df_loans.select_dtypes(include=['float16', 'float32', 'float64'])
for col in df_crunch:
    num_missing_vals = df_crunch[col].isnull().sum()
    print(col,  '=> Missing values: ', num_missing_vals)

company_permalink => Missing values:  0
company_name => Missing values:  0
company_category_code => Missing values:  52
company_country_code => Missing values:  0
company_state_code => Missing values:  53
company_region => Missing values:  0
company_city => Missing values:  64
investor_permalink => Missing values:  0
investor_name => Missing values:  0
investor_category_code => Missing values:  2557
investor_country_code => Missing values:  778
investor_state_code => Missing values:  1371
investor_region => Missing values:  0
investor_city => Missing values:  900
funding_round_type => Missing values:  0
funded_at => Missing values:  0
funded_month => Missing values:  0
funded_quarter => Missing values:  0
funded_year => Missing values:  0
raised_amount_usd => Missing values:  653


In [5]:
# Memory usage of columns
df_crunch = pd.read_csv('crunchbase-investments.csv', encoding_errors='ignore')
print(df_crunch.memory_usage(deep='True'))

<ipython-input-5-dcf6a03b8d6c>:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crunch = pd.read_csv('crunchbase-investments.csv', encoding_errors='ignore')


Index                         128
company_permalink         4057788
company_name              3590018
company_category_code     3421104
company_country_code      3172176
company_state_code        3106051
company_region            3411545
company_city              3505886
investor_permalink        4980548
investor_name             3914250
investor_category_code    1771304
investor_country_code     2836172
investor_state_code       2665487
investor_region           3395797
investor_city             3071603
funding_round_type        3410707
funded_at                 3542185
funded_month              3383584
funded_quarter            3383584
funded_year                422960
raised_amount_usd          422960
dtype: int64


In [6]:
# Memory usage across all the chunks

# Reset the iterable
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')

memory_used = 0

# Iterate over all the chunks
for df_loans in df_crunch_iter:
    memory_used += df_loans.memory_usage(deep=True).sum()

print('Total memory across chunks: ', memory_used)

# From the output below, memory across chunks is 59.7 MB

Total memory across chunks:  59751637


**Columns which may not be useful for analysis:**

The columns with many missing values will probably not be useful for analysis. These include: 
- company_category_code
- company_state_code
- investor_category_code
- investor_country_code
- investor_state_code
- investor_city
- raised_amount_usd

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


## Identify data types for each column

The types for each column can be seen from the output of the memory usage function above: the columns with type 'object' are strings (column 0 - 17). Column 18 and 19 are int and float, respectively, as the output shows.

The column types can also be gleaned from inspecting the actual csv file.

## Numeric columns we can represent using more space efficient types

Column 18 is a float64 and can be converted to an integer.

In [7]:
# Function to convert floats to integer
from pandas.api.types import is_numeric_dtype

def to_integers(df):
    for col in df:
        if is_numeric_dtype(df[col]):
            if df[col].isnull().sum() == 0:
                df[col] = df[col].astype('int')
                df[col] = pd.to_numeric(df[col], downcast='integer')

In [8]:
# Reset the iterable
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')
df_crunch = next(df_crunch_iter)

print('Before conversion: ', df_crunch.memory_usage(deep=True).sum())
to_integers(df_crunch)
print('After conversion: ', df_crunch.memory_usage(deep=True).sum())

# As the ouput below shows, changing col 18 from float to int achieves a slightly lower memory usage

Before conversion:  5849432
After conversion:  5819432


## Analyze unique value counts across all chunks

In [9]:
# Number of unique values in each string column

# Reset the iterable
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')

# Dictionary to keep track of value counts
crunch_val_count = {}
num_rows = 0

def count_unique(df):
    for col in df:
        num_unique = len(df[col].value_counts())
        if col not in crunch_val_count.keys():
            crunch_val_count[col] = num_unique
        else:
            crunch_val_count[col] += num_unique

# Count the unique values for all chunks
for df_crunch in df_crunch_iter:
    num_rows += len(df_crunch)

    df_crunch_string = df_crunch.select_dtypes(include=['object'])
    count_unique(df_crunch_string)


# count_unique(df_crunch_string)
print('Number of unique values for each column:\n')
for field, count in crunch_val_count.items():
    print(f'{field}: {count} (Ratio: {count / num_rows * 100:.2f}%)')

Number of unique values for each column:

company_permalink: 30586 (Ratio: 57.85%)
company_name: 30586 (Ratio: 57.85%)
company_category_code: 451 (Ratio: 0.85%)
company_country_code: 12 (Ratio: 0.02%)
company_state_code: 494 (Ratio: 0.93%)
company_region: 2066 (Ratio: 3.91%)
company_city: 5181 (Ratio: 9.80%)
investor_permalink: 10558 (Ratio: 19.97%)
investor_name: 10485 (Ratio: 19.83%)
investor_category_code: 33 (Ratio: 0.06%)
investor_country_code: 310 (Ratio: 0.59%)
investor_state_code: 327 (Ratio: 0.62%)
investor_region: 1337 (Ratio: 2.53%)
investor_city: 2038 (Ratio: 3.85%)
funding_round_type: 88 (Ratio: 0.17%)
funded_at: 18304 (Ratio: 34.62%)
funded_month: 1676 (Ratio: 3.17%)
funded_quarter: 656 (Ratio: 1.24%)


By inspecting the output from the cell above, we see that ALL the string columns except `company_permalink` and `company_name` have less than 50% unique values. The columns with less than 50% unique values can be optimized by converting to categorical types.

In [10]:
# Reset the iterable
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')
df_crunch = next(df_crunch_iter)
# df_crunch_string = df_crunch.select_dtypes(include=['object'])

def to_categorical(df):
    # num_rows is the number of rows in the chunk
    num_rows = df.shape[0]

    for col in df:
        if len(df[col].value_counts()) / num_rows < 0.5 and not is_numeric_dtype(df[col]):
            df[col] = df[col].astype('category')

# Testing to ascertain memory reduction
print('Before optimization: ', df_crunch.memory_usage(deep=True).sum())
to_categorical(df_crunch)
print('After optimization: ', df_crunch.memory_usage(deep=True).sum())

Before optimization:  5849432
After optimization:  1579992


The output from the cell above shows a significant reduction in memory for a single chunk.

## Clean some text columns and separate them into multiple numeric columns 

The funded_at column can be split into 3 columns, so that all of them are integers. After that, funded_month and funded_year are not useful since their information is contained in funded_at.

In [11]:
# Reset the iterable
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')
df_crunch = next(df_crunch_iter)

# Check size before operations
print('Before: ', df_crunch.memory_usage(deep=True).sum())

def clean_text_cols(df):
    #drop some unnecessary columns
    df.drop(['funded_month', 'funded_year'], axis=1, inplace=True)

    # Split the appropriate column
    df[['funded_year', 'funded_month', 'funded_day']] = df['funded_at'].str.split('-', expand=True)
    # Then drop the columns not necessary
    df.drop(['funded_at'], axis=1, inplace=True)

    # After the above, those three columns can be converted to integers

    for col in ['funded_year', 'funded_month', 'funded_day']:
        if df[col].isnull().sum() == 0:
            df[col] = df[col].astype('int')
            df[col] = pd.to_numeric(df_crunch[col], downcast='integer')

clean_text_cols(df_crunch)

print('After: ', df_crunch.memory_usage(deep=True).sum())

Before:  5849432
After:  5174432


## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [12]:
# First bring together everything done in the cells above into one place

def preprocess_chunk(df):
    # Numeric columns wiht more efficient representation
    to_integers(df)

    # Optimize string columns
    to_categorical(df)

    # Clean text columns
    clean_text_cols(df)

# Reset the iterable
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')
df_crunch = next(df_crunch_iter)

# Check size before and after preprocessing
print('Before: ', df_crunch.memory_usage(deep=True).sum())
preprocess_chunk(df_crunch)
print('After: ', df_crunch.memory_usage(deep=True).sum())

Before:  5849432
After:  1347201


## Create and connect to a new SQLite database file.

In [13]:
# Create and connect to a new SQLite database file
import sqlite3
conn = sqlite3.connect('crunchbase.db')

## Export each chunk to a new table in the SQLite database.

In [14]:
# Reset the iterable
df_crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding_errors='ignore')

for df_crunch in df_crunch_iter:
    preprocess_chunk(df_crunch)
    df_crunch.to_sql('crunchbase', conn, if_exists='append', index=False)

In [15]:
# Get a cursor object to query the sqlite db
cursor = conn.cursor()

# Check that the crunchbase table actually exists in the db
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
result = cursor.execute(sql_query)
print(result.fetchall())

[('crunchbase',)]


## Query the table 

In [16]:
# Query to check the data types of the columns in the sqlite db
sql_query = """PRAGMA table_info(crunchbase);"""
results = cursor.execute(sql_query).fetchall()
# Print the result
for result in results:
    print(f'{result[1]} => {result[2]}')
# The indices for result above were determined by first printing out to see 
# the index of the column name and data type

company_permalink => TEXT
company_name => TEXT
company_category_code => TEXT
company_country_code => TEXT
company_state_code => TEXT
company_region => TEXT
company_city => TEXT
investor_permalink => TEXT
investor_name => TEXT
investor_category_code => TEXT
investor_country_code => TEXT
investor_state_code => TEXT
investor_region => TEXT
investor_city => TEXT
funding_round_type => TEXT
funded_quarter => TEXT
raised_amount_usd => REAL
funded_year => INTEGER
funded_month => INTEGER
funded_day => INTEGER


The output from the cell shows that the query of the database using sql reveals that the datatypes are as expected following the prior optimizations that were done using pandas. For example, funded_year, funded_month and funded_day are integers, rather than having funded_at as a string.

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

## What proportion of the total amount of funds did the top/bottom 10% and 1% raise?

In [17]:
# First find the total amount of funds raised
query = "SELECT SUM(raised_amount_usd) AS total_amount_raised FROM crunchbase"
total_amount_raised = pd.read_sql(query, conn)
print(total_amount_raised, '\n')

# Amount raised by the various startups%
query = """SELECT company_name, investor_name, SUM(raised_amount_usd) AS raised_amount 
            FROM crunchbase GROUP BY investor_name
            HAVING raised_amount > 0
            ORDER BY raised_amount DESC"""
# The 'HAVING raised_amount > 0' has been introduced to eliminate the startups which have a blank
# for raised amount for any one of their investors.
startup_amounts = pd.read_sql(query, conn)
print(startup_amounts)

   total_amount_raised
0         6.817322e+11 

            company_name                                      investor_name  \
0               LifeLock                   Kleiner Perkins Caufield & Byers   
1         ChannelAdvisor                          New Enterprise Associates   
2               Facebook                                     Accel Partners   
3                   Veoh                                      Goldman Sachs   
4                  Joost                                    Sequoia Capital   
...                  ...                                                ...   
9810  PictureMe Universe  UW-Eau Claire Entrepreneur Program CEO Idea Ch...   
9811        TrustDegrees                                     Sean Barberous   
9812            uromovie                                                cnc   
9813  PictureMe Universe  UW-Eau Claire Office of Research and Sponsored...   
9814   Main Street Stark                                    Jeff SKI Kinsey   

   

From the output of the cell above, we have the total amount raised and the amount raised by each startup. We also see that there are 9815 startups which are listed (NB: these are the ones that have complete data). We can therefore get the amount of funds raised by the top 10% (top 982 rows) and top 1% (top 98 rows) by simple division, as follows.

In [18]:
# Proportion of funds raised by top 10%
proportion = 100 * sum(startup_amounts['raised_amount'][:982]) / total_amount_raised['total_amount_raised'][0]
print(f'Proportion raised by top 10%: {proportion:.2f}%')

# Proportion of funds raised by top 1%
proportion = 100 * sum(startup_amounts['raised_amount'][:98]) / total_amount_raised['total_amount_raised'][0]
print(f'Proportion raised by top 1%: {proportion:.2f}%')

Proportion raised by top 10%: 81.91%
Proportion raised by top 1%: 39.40%


In [19]:
# Proportion of funds raised by bottom 10%
proportion = 100 * sum(startup_amounts['raised_amount'][-982:]) / total_amount_raised['total_amount_raised'][0]
print(f'Proportion raised by bottom 10%: {proportion:.2f}%')

# Proportion of funds raised by bottom 1%
proportion = 100 * sum(startup_amounts['raised_amount'][-98:]) / total_amount_raised['total_amount_raised'][0]
print(f'Proportion raised by bottom 1%: {proportion:.2f}%')

Proportion raised by bottom 10%: 0.03%
Proportion raised by bottom 1%: 0.00%


## Which category of company attracted the most investments?

In [20]:
query = """
            SELECT company_category_code, COUNT(*) AS num_investments
            FROM crunchbase GROUP BY company_category_code 
            HAVING company_category_code != 'None' AND num_investments > 1
            ORDER BY num_investments DESC
            """
company_category_counts = pd.read_sql(query, conn)
print(company_category_counts, '\n')

   company_category_code  num_investments
0               software             7243
1                    web             5015
2                biotech             4951
3             enterprise             4489
4                 mobile             4067
5            advertising             3200
6              ecommerce             2168
7              cleantech             1948
8            games_video             1893
9              analytics             1863
10              hardware             1537
11               medical             1315
12         semiconductor             1292
13       network_hosting             1075
14              security              996
15               finance              931
16                social              920
17             education              783
18                health              670
19      public_relations              659
20                search              632
21                 other              481
22             messaging          

From the output of the cell above, it can be seen that **software** companies attracted the most investments.

## Which investor contributed the most money (across all startups)?

In [21]:
query = """
            SELECT investor_name, company_name, MAX(raised_amount_usd) AS amount_invested
            FROM crunchbase GROUP BY investor_name, company_name 
            ORDER BY amount_invested DESC
            """
most_per_startup = pd.read_sql(query, conn)
print(most_per_startup)

             investor_name      company_name  amount_invested
0              BrightHouse         Clearwire     3.200000e+09
1                  Comcast         Clearwire     3.200000e+09
2                   Google         Clearwire     3.200000e+09
3                    Intel         Clearwire     3.200000e+09
4              Time Warner         Clearwire     3.200000e+09
...                    ...               ...              ...
40963      reinmkr capital          GazeHawk              NaN
40964      reinmkr capital          Voicendo              NaN
40965      vSpring Capital  BioMicro Systems              NaN
40966  ventureblue Capital            Mangia              NaN
40967    venturecapital.de            zuuka!              NaN

[40968 rows x 3 columns]


From the output of the cell above we see that BrightHouse contributed the most money for a single startup; they contributied $3,200,000,000 to Clearwire.

In [22]:
print(most_per_startup['amount_invested'][0])

3200000000.0


## Which funding round was the most popular? Which was the least popular?

In [23]:
query = """
            SELECT funding_round_type, COUNT(*) AS occurences
            FROM crunchbase GROUP BY funding_round_type 
            HAVING funding_round_type != 'None'
            ORDER BY occurences DESC
            """
most_popular = pd.read_sql(query, conn)
print(most_popular, '\n')

  funding_round_type  occurences
0           series-a       13938
1          series-c+       10870
2              angel        8989
3            venture        8917
4           series-b        8794
5              other         964
6     private-equity         357
7           post-ipo          33
8       crowdfunding           5 



From the output of the cell above, we see that the **series-a round was most popular** and the **crowdfunding was least popular**